In [17]:
from osgeo import gdal
import numpy as np

# Path to your HDF file
file_path = 'downloaded_file.hdf'

# Open the HDF file
gdal_file = gdal.Open(file_path)

# Check the subdatasets (assuming the file has multiple datasets like most HDF files)
subdatasets = gdal_file.GetSubDatasets()

# Print subdatasets for reference
print("Subdatasets in the HDF file:")
for i, subdataset in enumerate(subdatasets):
    print(f"{i}: {subdataset[0]}")

# You need to identify the correct subdataset that contains EVI data.
# Typically, MODIS data has subdatasets like "MOD_Grid_BRDF:BRDF_Albedo_Parameter1", etc.
# You'll need to inspect your specific file to find the EVI dataset.
# For the sake of this example, we'll assume the EVI subdataset is at index 0.

# Open the EVI subdataset (you might need to change the index depending on your file)
evi_dataset = gdal.Open(subdatasets[1][0]) 
vi_quality_band = gdal.Open(subdatasets[2][0])
vi_quality_data = vi_quality_band.ReadAsArray()

# Extract specific bits
# Extract bits 0-1 for VI usefulness (use bitwise AND to isolate the bits, and shift them into position)
vi_usefulness = (vi_quality_data & 0b11)  # Bits [0-1]

# Extract bits 2-5 for Aerosol quantity (shift right by 2, then mask the 4 bits)
aerosol_quantity = (vi_quality_data >> 2) & 0b1111  # Bits [2-5]

# Extract bits 6-7 for Cloud adjacency (shift right by 6, then mask the 2 bits)
cloud_adj = (vi_quality_data >> 6) & 0b11  # Bits [6-7]

# Combine these into an 8-bit value by shifting the bits into place
# For example, combining usefulness and aerosol quantity into an 8-bit value
combined_quality = (cloud_adj << 6) | (aerosol_quantity << 2) | vi_usefulness  # 8-bit combined value

# Set NoData value for the quality band (optional, if there are invalid values)


Subdatasets in the HDF file:
0: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days NDVI"
1: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days EVI"
2: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days VI Quality"
3: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days red reflectance"
4: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days NIR reflectance"
5: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days blue reflectance"
6: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days MIR reflectance"
7: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days view zenith angle"
8: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY_250m_500m_VI:"250m 16 days sun zenith angle"
9: HDF4_EOS:EOS_GRID:"downloaded_file.hdf":MODIS_Grid_16DAY

In [24]:

output_tif = "evi_output.tif"

# Get driver for GeoTIFF format
driver = gdal.GetDriverByName("GTiff")

# Create the GeoTIFF file with the same dimensions and type as the original dataset
out_tif = driver.Create(output_tif, 
                        evi_dataset.RasterXSize, 
                        evi_dataset.RasterYSize, 
                        evi_dataset.RasterCount, 
                        evi_dataset.GetRasterBand(1).DataType)

# Set georeferencing information (projection and geotransform)
out_tif.SetProjection(evi_dataset.GetProjection())
out_tif.SetGeoTransform(evi_dataset.GetGeoTransform())

# Copy each band from the original dataset to the new GeoTIFF

in_band = evi_dataset.GetRasterBand(1)
out_band = out_tif.GetRasterBand(1)

# Copy data from original band to GeoTIFF band
out_band.WriteArray(in_band.ReadAsArray())

# Copy metadata
out_band.SetNoDataValue(in_band.GetNoDataValue())

out_tif.FlushCache()
out_tif = None

output_tif = "evi_quality.tif"

# Add a new band specifically for the quality data as GDT_Byte
out_tif = driver.Create(output_tif, 
                        evi_dataset.RasterXSize, 
                        evi_dataset.RasterYSize, 
                        1, 
                        gdal.GDT_Byte)
out_band = out_tif.GetRasterBand(1)

# Copy data from original band to GeoTIFF band
out_band.WriteArray(combined_quality.astype(np.uint8))

# Copy metadata
out_band.SetNoDataValue(in_band.GetNoDataValue())
out_band.FlushCache()
out_tif = None


3


GeoTransform: (-6671703.118, 231.65635826395825, 0.0, -2223901.039333, 0.0, -231.65635826395834)
Projection: PROJCS["unnamed",GEOGCS["Unknown datum based upon the custom spheroid",DATUM["Not specified (based on custom spheroid)",SPHEROID["Custom spheroid",6371007.181,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["Meter",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Top-left corner lat/lon: -2223901.039333, -6671703.118
bot-right corner lat/lon: -3335388.246283472, -5560215.911049528


Top-left corner lat/lon: -39.99999998988649, -59.99999999461182
Top-left corner lat/lon: -46.669277413437534, -50.00416666217315
